In [ ]:
import sys
sys.path.insert(0, '../')
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
from shapely.geometry import Point
from src.config import PARAM, PATH_RESOURCES, PATH_RESULTS
from src.geo_data import load_geonames

In [ ]:
map_ = geopandas.read_file(PATH_RESOURCES / 'shapefiles/nl_cbs_provincies_2018')
map_.crs

In [ ]:
df_geo_ = load_geonames()

In [ ]:
df_geo = df_geo_.query("country_code == 'NL'").copy()

In [ ]:
geometry = [Point(xy) for xy in zip(df_geo.longitude, df_geo.latitude)]
crs = {'init' :'epsg:4326'}
gdf_ = geopandas.GeoDataFrame(df_geo, crs=crs, geometry=geometry)
gdf_ = gdf_.to_crs({'init' :'epsg:28992'})

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
map_.plot(ax=ax, color='lightyellow', edgecolor='lightgray')
scatter = gdf_.plot(
    ax=ax, 
    marker='o', 
    color='black', 
    alpha=0.5, 
    markersize=2
    )
ax.set_title('Distributie plaatsnamen', fontdict={'fontsize':24})
plt.tight_layout()

In [ ]:
df_attr_counts = pd.read_pickle('../results/df_attribute_counts.pkl')
df = df_attr_counts.xs('places_nl', level=0)
df = df.append(df_attr_counts.xs('places_fr', level=0))
df = df.rename(columns={'index': 'naam'})

df_annotations = pd.read_pickle(PATH_RESULTS / 'df_annotations_places_nl.pkl')
df_annotations = df_annotations.append(pd.read_pickle(PATH_RESULTS / 'df_annotations_places_fr.pkl'))
df_annotations['positive'] = df_annotations.annotation == '+'
positives = df_annotations.groupby('phrase')['positive'].any()

df_ = df[df.index.isin(positives[positives].index)]

In [ ]:
gdf = gdf_.merge(df_, how='left', left_on='alternate_name', right_index=True)

In [ ]:
factor = 15
thrshld = 5
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(24, 28))
for ax, batch in zip(axs.reshape(-1), PARAM.lexisnexis.batches):
    map_.plot(ax=ax, color='lightyellow', edgecolor='lightgray')
    scatter = gdf.plot(
        ax=ax, 
        marker='o', 
        color='red', 
        alpha=0.5, 
        markersize=gdf[batch].fillna(0) * factor
        )
    kw = dict(
        prop="sizes",
        num=[1,10,50,200],
        color='red',
        alpha=0.5,
        func=lambda s: s/factor,
        )
    legend = ax.legend(
        *scatter.collections[1].legend_elements(**kw),
        loc="upper left",
        borderpad=2.0,
        labelspacing=4.5,
        handletextpad=2.0,
        frameon=False,
        )
    ax.set_title(batch, fontdict={'fontsize':24})
    
    rows = gdf.loc[gdf[batch] >= thrshld].dropna(subset=[batch])
    texts = [
        ax.text(
            row['geometry'].x, 
            row['geometry'].y, 
            row['alternate_name'], 
            ha='center', 
            va='center',
            fontsize=14,
            ) for _, row in rows.iterrows()
        ]
    adjust_text(texts, ax=ax)
plt.tight_layout()